# Create dictionary that can be saved as a .geojson map file with scalable markers

The idea is to print a dictionary structure that can then saved as a .geojson file, used to produce a simple map with scalable markers. In this case the markers will be 3 sided polygons, with one of the vertices pointing to the exact coordinates of the user given location(s). The scale of the markers will depend on the user given value. The function was created to be used with a pandas dataframe from which to extract both locations and values.

## Problem to solve
There's an especific problem where some websites can't display very elaborate maps like the ones created with the folium Python library, but can display Maps that come as .geojson files. Github.com is a website were folium maps can't be displayed but .geojson maps can. Geojson maps don't offer a wide variety of marker customization. For example, scalable markers that appear in different sizes depending on a value are not readily available in .geojson maps.

## Solution

The solution is a function that can create a dictionary with all the parameters that will allow the use of scalable markers in a map when the dictionary is saved as a .geojson file and read.
As an example, two pandas dataframes will be created. One containing countries as locations and the other one containing cities.

In [1]:
#Test dataframe:
import pandas as pd
import numpy as np

df_countries = pd.DataFrame({'Country':['Kenya','France','Israel','Mexico','India'],
                   'Value':np.random.randint(10,20,size=5)})

df_cities = pd.DataFrame({'City':['London','Paris','Oslo','Berlin','Amsterdam'],
                   'Value':np.random.randint(10,20,size=5)})

Function creation:

In [2]:
#Function to get a dictionary in the style of a .geojson file:
import json
import requests

def GeoJSONdict(df,df_value,locations,colors,scale=1.0):
    '''
    Function that prints a usable block of code to create a .geojson file.
    df: Pandas dataframe.
    df_value: column of the dataframe containing the values to plot.
    locations: list of locations by name. Case-insensitive. Can be country or city.
    colors: list of colors for the locations to be plotted in.
    scale: size of the polygons, type float or int. Default=1.0
    Prints block of code to create a .geojson file.
    The .geojson file will create a map that marks each location with a 3 sided 
    polygon, with one of its vertices set at the exact coordinates of the location.
    The size of each polygon is determined by the values used and the scale given. 
    Result can be copied and tested in the site http://geojson.io
    '''
    try:
        assert len(colors)>=len(locations), \
        'colors list must be same length or longer than locations list!'

        #class jsDict will be created assubclass of dict, to modify the way 
        #dictionaries are printed. In this case it will allow for double quotes 
        #to be printed instead of single quotes. 
        class jsDict(dict):
            def __str__(self):
                return json.dumps(self)  

        coordinates = []#Coordinates in [lat,lon] format   
        for p in locations:#Iterate over list of location names
           #Use url to get country locations by inserting the country name
            url = '{0}{1}{2}'.format('https://nominatim.openstreetmap.org/search.php?q=',
                                     p,
                                     '&format=json&polygon=0')

            response = requests.get(url).json()[0]
            lst = [response.get(key) for key in ['lat','lon']]
            output = [float(i) for i in lst]
            coordinates.append(output)

        d = {'type': 'FeatureCollection',
              'features': []}

        for i in range(len(df)):

            point_move = df[df_value].values[i]*scale

            d['features'].append(dict(type='Feature',
                                      properties={'stroke':colors[i],
                                                  'stroke-width':2,
                                                  'stroke-opacity':1,
                                                  'fill':colors[i],
                                                  'fill-opacity': 0.65,
                                                  'Location':locations[i],
                                                  'Value':'{:.3f}'.format(df[df_value].values[i])},
                                      geometry={'type':'Polygon',
                                                'coordinates':[[[coordinates[i][1],coordinates[i][0]],
                                      [coordinates[i][1]+point_move,coordinates[i][0]+point_move],
                                      [coordinates[i][1]-point_move,coordinates[i][0]+point_move],
                                      [coordinates[i][1],coordinates[i][0]]                          
                                                               ]]}))
        code = jsDict(d) 
        print(json.dumps(code,indent=2))#Use json.dumps to indent printed version of dictionary.
    except NameError as nerr:
        print('NameError: {} \nPlease '.format(nerr)+
        'import json library and requests library!')

Function call with df_countries as an argument:

In [3]:
colors = ['#e6194b','#3cb44b','#641e16','#4363d8','#f58231']#set a colors variable

GeoJSONdict(df_countries,'Value',df_countries['Country'],colors,scale=1)

{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {
        "stroke": "#e6194b",
        "stroke-width": 2,
        "stroke-opacity": 1,
        "fill": "#e6194b",
        "fill-opacity": 0.65,
        "Location": "Kenya",
        "Value": "10.000"
      },
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              38.4313975,
              1.4419683
            ],
            [
              48.4313975,
              11.4419683
            ],
            [
              28.431397500000003,
              11.4419683
            ],
            [
              38.4313975,
              1.4419683
            ]
          ]
        ]
      }
    },
    {
      "type": "Feature",
      "properties": {
        "stroke": "#3cb44b",
        "stroke-width": 2,
        "stroke-opacity": 1,
        "fill": "#3cb44b",
        "fill-opacity": 0.65,
        "Location": "France",
        "Value": "15.

All printed outputs of the function can be copied and pasted at the [geojson.io](http://geojson.io) website to check the result. When the output of GeoJSONdict() is used to create a .geojson file with country locations, the following map can be read from that file:

<img src="countries_map.png" width="600">

Function call with df_cities as an argument:

In [4]:
GeoJSONdict(df_cities,'Value',df_cities['City'],colors,scale=0.2)

{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {
        "stroke": "#e6194b",
        "stroke-width": 2,
        "stroke-opacity": 1,
        "fill": "#e6194b",
        "fill-opacity": 0.65,
        "Location": "London",
        "Value": "14.000"
      },
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -0.1276474,
              51.5073219
            ],
            [
              2.6723526000000004,
              54.3073219
            ],
            [
              -2.9276474,
              54.3073219
            ],
            [
              -0.1276474,
              51.5073219
            ]
          ]
        ]
      }
    },
    {
      "type": "Feature",
      "properties": {
        "stroke": "#3cb44b",
        "stroke-width": 2,
        "stroke-opacity": 1,
        "fill": "#3cb44b",
        "fill-opacity": 0.65,
        "Location": "Paris",
        "Value": "1

GeoJSONdict() with df_cities as an argument generated an output that can be used to construct a .geojson file which produces the map below: 
<img src="cities_map.png" width="500">